In [1]:
import pandas as pd
import numpy as np

In [19]:
df = pd.read_csv('formula1_daten.csv', sep = ';', decimal = '.')
del df['Unnamed: 0']
df.shape

(157755, 20)

Aufteilen des Datensatzes nach einzelnen Rennen und verkürzen der Datensätze nach prozentualem Rennfortschritt, zusätzlich wird eine Spalte hinzugefügt, die pro Runde die kumulierten Millisekunden enthält

In [20]:
#dictionary enthält einzelne datensätze zu jedem rennen
split_by_race = {}
#grenzwert ab dem vorhersage über rennausgang gemacht werden soll (orientiert sich an race_completion)
border = 0.5
#dictionary, welches renndatensätze nur bis zu einem gewissen zeitpunkt enthält (zp wird nach border gewählt)
sliced_races = {}

for rid in df['raceId'].unique():
    race = df.where(df.raceId == rid).dropna(how = 'all')
    race.reset_index(inplace = True, drop = True)
    split_by_race[rid] = race
    
    #finden der lap_number wo race_completion die angegebene border überschreitet
    last_lap_num = race.where(race.race_completion == min(race.where(race.race_completion >= border).dropna(how = 'all')['race_completion'])).dropna(how = 'all')['lap_number'].unique()[0]
    
    #es werden nur daten aus race genommen, die bis zu dieser lap_number gehen
    race_shortened = race.where(race.lap_number < last_lap_num).dropna(how = 'all')
    race_shortened.reset_index(inplace = True, drop = True)
    
    #hinzufügen einer spalte die die insgesamt gefahrenen millisekunden enthält bis zu dieser Runde pro Fahrer enthält
    race_shortened['sum_milliseconds_pro_lap'] = 0
    
    for did in race_shortened.driverId.unique():
        
        l_lap_driver = race_shortened.where(race_shortened.driverId == did).dropna(how = 'all')['lap_number'].tail(1).reset_index()['lap_number'][0]
        status_driver = race_shortened.status_clean.unique()[0]
        if status_driver == 'DNF':
            if l_lap_driver == last_lap_num-1:#der Fahrer ist das Rennen bis zu dieser Runde ohne DNF gefahren
                idces = race_shortened.where(race_shortened.driverId == did).dropna(how = 'all').index
                idces = list(idces)
                idx1 = idces[0]
                idx2 = idces[-1]
                race_shortened.loc[idx1:idx2, 'status_clean'] = 'No_DNF' #DNF wird als No_DNF überschrieben, da bist zu dieser Runde kein DNF stattgefunden hat
                race_shortened['status_binär'] = [1 if race_shortened.at[v,'status_clean']== 'No_DNF' else 0 for v in range(len(race_shortened.status_clean))]
        for lapnum in race_shortened.lap_number.unique():
            sum_ms = np.sum(race_shortened.where(np.logical_and(race_shortened.driverId == did,race_shortened.lap_number<=lapnum)).dropna(how = 'all')['lap_in_milliseconds'])
            
            #setzen der bisher gefahrenen Zeit (kumuliert) pro Fahrer und Runde
            race_shortened.loc[race_shortened.where(np.logical_and(race_shortened.driverId == did,race_shortened.lap_number==lapnum)).dropna(how = 'all').index,'sum_milliseconds_pro_lap'] = sum_ms
        race_shortened['sum_minutes_pro_lap'] = race_shortened['sum_milliseconds_pro_lap'] / 60000
    
    sliced_races[rid] = race_shortened
    
    
    

In [21]:
sliced_races[841]

,raceId,year,circuitId,grandprix_name,driverId,lap_number,lap_position,lap_in_milliseconds,driver_fullname,podium_position,...,constructor_name,stop_binary,total_laps,race_completion,grid,status_clean,total_milliseconds,status_binär,form,sum_milliseconds_pro_lap
0,841.0,2011.0,1.0,Australian Grand Prix,1.0,1.0,2.0,100573.0,Lewis Hamilton,2.0,...,McLaren,0.0,58.0,0.017241,2.0,No_DNF,5.392556e+06,1.0,0.0,100573.0
1,841.0,2011.0,1.0,Australian Grand Prix,1.0,2.0,2.0,93774.0,Lewis Hamilton,2.0,...,McLaren,0.0,58.0,0.034483,2.0,No_DNF,5.392556e+06,1.0,0.0,194347.0
2,841.0,2011.0,1.0,Australian Grand Prix,1.0,3.0,2.0,92900.0,Lewis Hamilton,2.0,...,McLaren,0.0,58.0,0.051724,2.0,No_DNF,5.392556e+06,1.0,0.0,287247.0
3,841.0,2011.0,1.0,Australian Grand Prix,1.0,4.0,2.0,92582.0,Lewis Hamilton,2.0,...,McLaren,0.0,58.0,0.068966,2.0,No_DNF,5.392556e+06,1.0,0.0,379829.0
4,841.0,2011.0,1.0,Australian Grand Prix,1.0,5.0,2.0,92471.0,Lewis Hamilton,2.0,...,McLaren,0.0,58.0,0.086207,2.0,No_DNF,5.392556e+06,1.0,0.0,472300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568,841.0,2011.0,1.0,Australian Grand Prix,816.0,24.0,17.0,97968.0,JÌ©rÌ«me d'Ambrosio,14.0,...,Virgin,0.0,58.0,0.413793,22.0,DNF,5.772630e+06,0.0,0.0,2421653.0
569,841.0,2011.0,1.0,Australian Grand Prix,816.0,25.0,17.0,99762.0,JÌ©rÌ«me d'Ambrosio,14.0,...,Virgin,0.0,58.0,0.431034,22.0,DNF,5.772630e+06,0.0,0.0,2521415.0
570,841.0,2011.0,1.0,Australian Grand Prix,816.0,26.0,17.0,98298.0,JÌ©rÌ«me d'Ambrosio,14.0,...,Virgin,0.0,58.0,0.448276,22.0,DNF,5.772630e+06,0.0,0.0,2619713.0
571,841.0,2011.0,1.0,Australian Grand Prix,816.0,27.0,17.0,98654.0,JÌ©rÌ«me d'Ambrosio,14.0,...,Virgin,0.0,58.0,0.465517,22.0,DNF,5.772630e+06,0.0,0.0,2718367.0


Anlegen von Ordnern, in denen DataFrames abgespeichert werden, um Zelle oben nicht immer ausführen zu müssen

In [23]:
import os

In [25]:
if not os.path.exists('sliced_data_50_neueForm'):
    os.makedirs('sliced_data_50_neueForm')
    
if not os.path.exists('split_data_50_neueForm'):
    os.makedirs('split_data_50_neueForm')

for key, value in sliced_races.items():
    name = 'sliced_data_50_neueForm/sliced_'+str(int(key))+'.csv'
    value.to_csv(name,sep = ';', decimal = '.')
for key, value in split_by_race.items():
    name = 'split_data_50_neueForm/split_'+str(int(key))+'.csv'
    value.to_csv(name,sep = ';', decimal = '.')

Laden der oben erzeugten DataFrames aus den erstellten Ordnern

In [ ]:
if os.path.exists('sliced_data'):
    csv_filenames = []
    #auslesen aller csv file dateinamen aus formula 1 datensatz und abspeichern in liste
    for filename in os.listdir(os.getcwd()+'/sliced_data'):
        typ = filename.split('.')[-1]
        name = filename.split('.')[0]
        if typ == 'csv':
            csv_filenames.append(filename)
    sliced_races = {}
    #einlesen und abspeichern als dataframe aller dateien
    for file in csv_filenames:
        try:
            df = pd.read_csv('sliced_data/'+file, engine = 'python', sep = ';', decimal = '.')
            del df['Unnamed: 0']
        except Exception as e:
            df = pd.read_csv('sliced_data/'+file, engine = 'c', sep = ';', decimal = '.')
            del df['Unnamed: 0']
            print(e)
        #print(df.head())
        f = int(file.split('_')[-1].split('.')[0])
        sliced_races[f] = df
    print('Einlesen der sliced Dateien erfolgreich')
else:
    print('Dateien können nicht eingelesen werden, da kein entsprechendes Verzeichnis existiert!')
    
if os.path.exists('split_data'):
    csv_filenames = []
    #auslesen aller csv file dateinamen aus formula 1 datensatz und abspeichern in liste
    for filename in os.listdir(os.getcwd()+'/split_data'):
        typ = filename.split('.')[-1]
        name = filename.split('.')[0]
        if typ == 'csv':
            csv_filenames.append(filename)
    split_by_race = {}
    #einlesen und abspeichern als dataframe aller dateien
    for file in csv_filenames:
        try:
            df = pd.read_csv('split_data/'+file, engine = 'python', sep = ';', decimal = '.')
            del df['Unnamed: 0']
        except Exception as e:
            df = pd.read_csv('split_data/'+file, engine = 'c', sep = ';', decimal = '.')
            del df['Unnamed: 0']
            print(e)
        #print(df.head())
        f = int(file.split('_')[-1].split('.')[0])
        split_by_race[f] = df
    print('Einlesen der split Dateien erfolgreich')
else:
    print('Dateien können nicht eingelesen werden, da kein entsprechendes Verzeichnis existiert!')

alle Daten werden in sliced_races, split_by_race gespeichert

In [20]:
sliced_races[841]['race_completion']

0      0.017241
1      0.034483
2      0.051724
3      0.068966
4      0.086207
         ...   
478    0.327586
479    0.344828
480    0.362069
481    0.379310
482    0.396552
Name: race_completion, Length: 483, dtype: float64

#### Anhang:

In [11]:
temp = split_by_race[841]

In [ ]:
#setzen der prozentualen Grenze, ab der weiterer Verlauf des Rennens vorhergesagt werden soll
border = 0.4

for race in split_by_race.values():
    
    #finden der lap_number wo race_completion die angegebene border überschreitet
    last_lap_num = race.where(race.race_completion == min(race.where(race.race_completion >= border).dropna(how = 'all')['race_completion'])).dropna(how = 'all')['lap_number'].unique()[0]
    
    #es werden nur daten aus race genommen, die bis zu dieser lap_number gehen
    race_shortened = race.where(race.lap_number < last_lap_num).dropna(how = 'all')
    race_shortened.reset_index(inplace = True, drop = True)
    

In [16]:
temp.where(temp.race_completion == min(temp.where(temp.race_completion >= 0.4).dropna(how = 'all')['race_completion'])).dropna(how = 'all')['lap_number'].unique()[0]

24.0